In [1]:
import torch
from torch import nn

In [2]:
def nin_block(output_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.LazyConv2d(output_channels, kernel_size,
                      stride=strides, padding=padding),
        nn.ReLU(),
        nn.LazyConv2d(output_channels, kernel_size=1), nn.ReLU(),
        nn.LazyConv2d(output_channels, kernel_size=1), nn.ReLU(),
    )

In [3]:
from d2l_common import Classifier

class NiN(Classifier):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nin_block(96, kernel_size=11, strides=4, padding=0),
            nn.MaxPool2d(3, stride=2),
            nin_block(256, kernel_size=5, strides=1, padding=2),
            nn.MaxPool2d(3, stride=2),
            nin_block(384, kernel_size=3, strides=1, padding=1),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout(p=0.5),
            nin_block(num_classes, kernel_size=3, strides=1, padding=1),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten()
        )
        

In [5]:
model = NiN()
model.layer_summary((1, 1, 224, 224))

/Users/tangyun/opt/miniconda3/envs/rl/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [6]:
from d2l_common import Trainer, FasionMNIST
trainer = Trainer(max_epochs=10)
data = FasionMNIST(batch_size=128, resize=(224, 224))
# model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data)

AssertionError: no gpu supported yet